In [1]:
import sys
sys.path.insert(0, '../..')

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import edunn as nn
from edunn import utils

In [3]:
np.set_printoptions(threshold=sys.maxsize)

# Capa Batch Normalization

La normalización por lotes aborda el problema de la inicialización deficiente de las redes neuronales. Se puede interpretar como hacer un preprocesamiento en cada capa de la red. Obliga a las activaciones en una red a adoptar una distribución gaussiana unitaria al comienzo del entrenamiento. Esto asegura que todas las neuronas tengan aproximadamente la misma distribución de salida en la red y mejora la tasa de convergencia.

Explicar el por qué la distribución de las activaciones en una red importa excede el propósito de estas guías, pero de ser de tu interés podés referirte a las páginas 46 — 62 en las [diapositivas de la conferencia](http://cs231n.stanford.edu/slides/2019/cs231n_2019_lecture07.pdf) ofrecidas por el curso de la universidad de Stanford.

## Método Forward

Digamos que tenemos un lote de activaciones $x$ en una capa, la versión de $x$ con media cero y varianza unitaria $\hat{x}$ es:

$$\hat{x}^{(k)}=\frac{x^{(k)}-\mathbb{E}[x^{(k)}]}{\sqrt{\text{Var}[x^{(k)}]}}$$

Esta es en realidad una operación diferenciable, por eso podemos aplicar la normalización por lotes en el entrenamiento.

El cálculo de ésta se resume en computar la media $\mu_\mathcal{B}$ y varianza $\sigma_\mathcal{B}^2$ de un mini-batch de $\mathcal{B}=\{x_1, \dots, x_N\}$. Los parámetros aprendibles de la capa son $\gamma$ y $\beta$ que son utilizados para escalar y desplazar los valores normalizados.

$$
\begin{aligned}
\mu_\mathcal{B} &= \frac{1}{N} \sum_{i=1}^{N} x_i & \text{(mini-batch mean)} \\
\sigma_\mathcal{B}^2 &= \frac{1}{N} \sum_{i=1}^{N} (x_i - \mu_\mathcal{B})^2 & \text{(mini-batch variance)} \\
\hat{x}_i &= \frac{x_i - \mu_\mathcal{B}}{\sqrt{\sigma_\mathcal{B}^2+\epsilon}} & \text{(normalize)} \\
\text{\textbf{BN}}_{\gamma,\beta}(x_i) &\stackrel{\text{def}}{=} \gamma \hat{x}_i + \beta = y_i & \text{(scale and shift)} 
\end{aligned}
$$

> NOTA: En la implementación, insertamos la capa `BatchNorm` justo después de una capa `Dense` o una capa `Conv2d`, y antes de las capas no lineales.


In [4]:
np.random.seed(123)

din=10
batch_size=2

x = np.random.rand(batch_size,din)
w = np.random.rand(din)
b = np.random.rand(din)
                       
gamma_initializer = nn.initializers.Constant(w)
beta_initializer = nn.initializers.Constant(b)

layer=nn.BatchNorm(num_features=din, gamma_initializer=gamma_initializer, beta_initializer=beta_initializer)

In [5]:
x

array([[0.69646919, 0.28613933, 0.22685145, 0.55131477, 0.71946897,
        0.42310646, 0.9807642 , 0.68482974, 0.4809319 , 0.39211752],
       [0.34317802, 0.72904971, 0.43857224, 0.0596779 , 0.39804426,
        0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759]])

In [6]:
y = layer.forward(x)
y, y.shape

(array([[ 0.72640427, -0.41564403, -0.29326955,  1.10465806,  1.14813386,
         -0.01063257,  0.78812861,  1.1216348 ,  0.65271203, -0.12849391],
        [-0.54219439,  1.28304638,  1.15499507, -0.11728786, -0.29647328,
          0.63515501,  0.06457401,  0.66514353,  1.23560801,  1.13216726]]),
 (2, 10))

## Método Backward

### `δEδβ`

El cálculo de los gradientes del error $E$ con respecto al parámetro $\beta$ se puede hacer derivando parcialmente como se explicó en guías anteriores:

$$
\frac{\partial E}{\partial \beta} = \frac{\partial E}{\partial y} \cdot \frac{\partial y}{\partial \beta}
$$

Como $y$ es un vector de $N$ elementos, tenemos que sumar por todos sus valores para aplicar la regla de la cadena:

$$
\frac{\partial E}{\partial \beta} = \frac{\partial E}{\partial y_1} \cdot \frac{\partial y_1}{\partial \beta} + \cdots + \frac{\partial E}{\partial y_N} \cdot \frac{\partial y_N}{\partial \beta} 
\qquad \text{donde} \qquad
\frac{\partial y_i}{\partial \beta} = \frac{\partial (\gamma \hat{x}_i + \beta)}{\partial \beta} = 1
$$

de este modo:

$$
\frac{\partial E}{\partial \beta} = \sum\limits_{i=1}^N \frac{\partial E}{\partial y_i} \cdot 1
$$

### `δEδγ`

El cálculo de los gradientes del error $E$ con respecto al parámetro $\gamma$ se puede hacer derivando parcialmente como se explicó en guías anteriores:

$$
\frac{\partial E}{\partial \gamma} = \frac{\partial E}{\partial y} \cdot \frac{\partial y}{\partial \gamma}
$$

Como $y$ es un vector de $N$ elementos, tenemos que sumar por todos sus valores para aplicar la regla de la cadena:

$$
\frac{\partial E}{\partial \gamma} = \frac{\partial E}{\partial y_1} \cdot \frac{\partial y_1}{\partial \gamma} + \cdots + \frac{\partial E}{\partial y_N} \cdot \frac{\partial y_N}{\partial \gamma} 
\qquad \text{donde} \qquad
\frac{\partial y_i}{\partial \gamma} = \frac{\partial (\gamma \hat{x}_i + \beta)}{\partial \gamma} = \hat{x}_i
$$

de este modo:

$$
\frac{\partial E}{\partial \gamma} = \sum\limits_{i=1}^N \frac{\partial E}{\partial y_i} \cdot \hat{x}_i
$$

### `δEδx`

<!-- Utilizando la regla de la cadena para el cálculo diferencial, esta nos dice que la derivada de una función compuesta es el producto de las derivadas de las funciones que la componen. -->

Teniendo en cuenta de qué depende cada función:

<center>

||||
|:-:|:-:|:-:|
|$E(y)$|$y(\hat{x},\gamma,\beta)$|$\hat{x}(\mu,\sigma^2,x)$|

</center>

Obtenemos que:

$$
\dfrac{\partial E}{\partial x_i} = \frac{\partial E}{\partial \hat{x}_i} \cdot \frac{\partial \hat{x}_i}{\partial x_i} + \frac{\partial E}{\partial \mu} \cdot \frac{\partial \mu}{\partial x_i} + \frac{\partial E}{\partial \sigma^2} \cdot \frac{\partial \sigma^2}{\partial x_i}
$$

En las siguientes subsecciones calcularemos la expresión correspondiente para el gradiente de cada componente.

#### `δEδx̂`

$$
\frac{\partial E}{\partial \hat{x}_i} = \frac{\partial E}{\partial y_i} \cdot \frac{\partial y_i}{\partial \hat{x}_i} = \frac{\partial E}{\partial y_i} \cdot \frac{\partial (\gamma \hat{x}_i + \beta)}{\partial \hat{x}_i} = \frac{\partial E}{\partial y_i} \cdot \gamma
$$

#### `δEδμ`

Notar que $\sigma^2$ se puede escribir en función de $\mu$, es por ello que $E$ depende de $\mu$ a traves de dos variables: $\hat{x}_i$​ y $\sigma^2$.

$$
\dfrac{\partial E}{\partial \mu} = \frac{\partial E}{\partial \hat{x}_i} \cdot \frac{\partial \hat{x}_i}{\partial \mu} + \frac{\partial E}{\partial \sigma^2} \cdot \frac{\partial \sigma^2}{\partial\mu}
$$

Calculando las derivadas parciales para `δx̂δμ` y `δσ²δμ`:

$$
\begin{aligned}
\hat{x}_i = \frac{(x_i - \mu)}{\sqrt{\sigma^2 + \epsilon}}
&\qquad \Rightarrow \qquad
\dfrac{\partial \hat{x}_i}{\partial \mu} = \frac{-1}{\sqrt{\sigma^2 + \epsilon}} \\
\sigma^2 = \frac{1}{N} \sum\limits_{i=1}^N (x_i - \mu)^2
&\qquad \Rightarrow \qquad
\dfrac{\partial \sigma^2}{\partial \mu} = \frac{1}{N} \sum\limits_{i=1}^N -2 \cdot (x_i - \mu) \\
\end{aligned}
$$

Reemplazando éstas últimas y dejando como variables a los gradientes del error $E$, obtenemos:

$$
\begin{aligned}
\frac{\partial E}{\partial \mu} &= \bigg(\sum\limits_{i=1}^N  \frac{\partial E}{\partial \hat{x}_i} \cdot \frac{-1}{\sqrt{\sigma^2 + \epsilon}} \bigg) + \bigg( \frac{\partial E}{\partial \sigma^2} \cdot \frac{1}{N} \sum\limits_{i=1}^N -2(x_i - \mu)   \bigg) \qquad \\
&= \bigg(\sum\limits_{i=1}^N  \frac{\partial E}{\partial \hat{x}_i} \cdot \frac{-1}{\sqrt{\sigma^2 + \epsilon}} \bigg) + \bigg( \frac{\partial E}{\partial \sigma^2} \cdot (-2) \cdot \bigg( \frac{1}{N} \sum\limits_{i=1}^N x_i - \frac{1}{N} \sum\limits_{i=1}^N \mu   \bigg) \bigg) \qquad \\
&= \bigg(\sum\limits_{i=1}^N  \frac{\partial E}{\partial \hat{x}_i} \cdot \frac{-1}{\sqrt{\sigma^2 + \epsilon}} \bigg) + \bigg( \frac{\partial E}{\partial \sigma^2} \cdot (-2) \cdot \underbrace{\bigg( \mu - \frac{N \cdot \mu}{N} \bigg)}_{0} \bigg) \qquad \\
&= \sum\limits_{i=1}^N  \frac{\partial E}{\partial \hat{x}_i} \cdot \frac{-1}{\sqrt{\sigma^2 + \epsilon}} \qquad \\
\end{aligned}
$$

#### `δEδσ²`

$$\frac{\partial E}{\partial \sigma^2} = \frac{\partial E}{\partial \hat{x}} \cdot \frac{\partial \hat{x}}{\partial \sigma^2}$$

Reescribiendo $\hat{x}_i$ para que su derivada sea más fácil de calcular, vemos que $(x_i - \mu)$ pasa a ser un factor constante, de modo que:

$$
\hat{x}_i = (x_i - \mu)(\sigma^2 + \epsilon)^{-0.5}
\qquad \Rightarrow \qquad
\dfrac{\partial \hat{x}}{\partial \sigma^2} = -0.5 \sum\limits_{i=1}^N (x_i - \mu) \cdot (\sigma^2 + \epsilon)^{-1.5}
$$

#### `δEδx` (cont.)

Calculando las derivadas parciales restantes (`δx̂δx`, `δμδx` y `δσ²δx`) de la expresión original obtenemos que:

<center>

||||
|:-:|:-:|:-:|
|$\dfrac{\partial \hat{x}_i}{\partial x_i} = \dfrac{1}{\sqrt{\sigma^2 + \epsilon}}$|$\dfrac{\partial \mu}{\partial x_i} = \dfrac{1}{N}$|$\dfrac{\partial \sigma^2}{\partial x_i} = \dfrac{2(x_i - \mu)}{N}$|

</center>

Finalmente podemos calcular el gradiente del error $E$ con respecto a $x$ utilizando el siguiente truco:

$$
(\sigma^2 + \epsilon)^{-1.5} = (\sigma^2 + \epsilon)^{-0.5}(\sigma^2 + \epsilon)^{-1} = (\sigma^2 + \epsilon)^{-0.5} \frac{1}{\sqrt{\sigma^2 + \epsilon}}\frac{1}{\sqrt{\sigma^2 + \epsilon}}
$$

de este modo:

$$
\begin{aligned}
\frac{\partial E}{\partial x_i} &= \bigg(\frac{\partial E}{\partial \hat{x}_i} \cdot \dfrac{1}{\sqrt{\sigma^2 + \epsilon}} \quad\; \bigg) + \bigg(\frac{\partial E}{\partial \mu} \cdot \dfrac{1}{N} \qquad\qquad\qquad\!\! \bigg) + \bigg(\frac{\partial E}{\partial \sigma^2} \cdot \dfrac{2(x_i - \mu)}{N}\bigg) \qquad \\
&= \bigg(\frac{\partial E}{\partial \hat{x}_i} \cdot \dfrac{1}{\sqrt{\sigma^2 + \epsilon}} \quad\; \bigg) + \bigg(\frac{1}{N} \sum\limits_{j=1}^N  \frac{\partial E}{\partial \hat{x}_j} \cdot \frac{-1}{\sqrt{\sigma^2 + \epsilon}}\bigg) + \bigg(-0.5 \sum\limits_{j=1}^N \frac{\partial E}{\partial \hat{x}_j} \cdot (x_j - \mu) \cdot (\sigma^2 + \epsilon)^{-1.5} \cdot \dfrac{2(x_i - \mu)}{N} \bigg) \qquad \\
&= \bigg(\frac{\partial E}{\partial \hat{x}_i} \cdot (\sigma^2 + \epsilon)^{-0.5} \bigg) - \bigg(\frac{(\sigma^2 + \epsilon)^{-0.5}}{N} \sum\limits_{j=1}^N  \frac{\partial E}{\partial \hat{x}_j} \;\, \bigg) - \bigg(\frac{(\sigma^2 + \epsilon)^{-0.5}}{N} \cdot \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}} \sum\limits_{j=1}^N \frac{\partial E}{\partial \hat{x}_j} \cdot \frac{(x_j - \mu)}{\sqrt{\sigma^2 + \epsilon}} \bigg )\qquad \\
&= \bigg(\frac{\partial E}{\partial \hat{x}_i} \cdot (\sigma^2 + \epsilon)^{-0.5} \bigg) - \bigg(\frac{(\sigma^2 + \epsilon)^{-0.5}}{N} \sum\limits_{j=1}^N  \frac{\partial E}{\partial \hat{x}_j} \;\, \bigg) - \bigg(\frac{(\sigma^2 + \epsilon)^{-0.5}}{N} \cdot \hat{x}_i \sum\limits_{j=1}^N \frac{\partial E}{\partial \hat{x}_j} \cdot \hat{x}_j \bigg )\qquad \\
&= \boxed{\frac{(\sigma^2 + \epsilon)^{-0.5}}{N} \bigg [N \frac{\partial E}{\partial \hat{x}_i} - \sum\limits_{j=1}^N  \frac{\partial E}{\partial \hat{x}_j} - \hat{x}_i \sum\limits_{j=1}^N \frac{\partial E}{\partial \hat{x}_j} \cdot \hat{x}_j\bigg ]} \qquad \\
&= \frac{(\sigma^2 + \epsilon)^{-0.5}}{N} \bigg [N \frac{\partial E}{\partial y_i} \cdot \gamma - \sum\limits_{j=1}^N  \frac{\partial E}{\partial y_j} \cdot \gamma - \hat{x}_i \sum\limits_{j=1}^N \frac{\partial E}{\partial y_j} \cdot \gamma \cdot \hat{x}_j\bigg ] \qquad \\
\end{aligned}
$$


In [7]:
# Define el gradiente de la salida
g = np.random.rand(*y.shape)

# Propaga el gradiente hacia atrás a través de la convolución
layer_grad = layer.backward(g)
layer_grad

(array([[ 2.89510703e-04, -2.77848552e-04, -8.71412370e-04,
         -2.67810219e-05,  4.55331949e-04, -2.21904874e-05,
          1.87777622e-06,  2.10144722e-05, -3.15033262e-02,
          9.45501401e-04],
        [-2.89510703e-04,  2.77848552e-04,  8.71412370e-04,
          2.67810219e-05, -4.55331949e-04,  2.21904874e-05,
         -1.87777622e-06, -2.10144722e-05,  3.15033262e-02,
         -9.45501401e-04]]),
 {'w': array([ 0.50324365,  0.71064996,  0.28564723, -0.13023102,  0.523444  ,
          0.0536546 ,  0.06600998,  0.30423557,  0.35322534, -0.10236735]),
  'b': array([0.74458162, 0.9419592 , 0.92034561, 0.95989422, 1.20907299,
         0.55457615, 0.90005648, 1.66686055, 1.39494196, 1.12331686])})

## Comprobaciones con PyTorch

In [8]:
import torch
import torch.nn as tnn

w = torch.from_numpy(w).to(torch.float)
b = torch.from_numpy(b).to(torch.float)
x = torch.from_numpy(x).to(torch.float)

batch_norm = tnn.BatchNorm1d(din)
batch_norm.weight.data = w
batch_norm.bias.data = b

x.requires_grad = True
batch_norm.weight.requires_grad = True
batch_norm.bias.requires_grad = True

y_torch = batch_norm(x)
y_torch

tensor([[ 0.7264, -0.4156, -0.2933,  1.1047,  1.1481, -0.0106,  0.7881,  1.1216,
          0.6527, -0.1285],
        [-0.5422,  1.2830,  1.1550, -0.1173, -0.2965,  0.6352,  0.0646,  0.6651,
          1.2356,  1.1322]], grad_fn=<NativeBatchNormBackward0>)

In [9]:
utils.check_same(y_torch.detach().numpy(),y,tol=1e-5)

SUCCESS :) Arrays are equal (tolerance 1e-05)


In [10]:
# Define el gradiente de la salida
g = torch.from_numpy(g).to(torch.double)

# Propaga el gradiente hacia atrás
x.grad = None  # Limpiamos los gradientes existentes
y_torch.backward(g)

# Imprime el gradiente de la imagen de entrada, gamma y beta
print("Gradiente de la entrada (δE/δx):")
print(x.grad, x.grad.shape)
print("\nGradiente de gamma (δE/δγ):")
print(batch_norm.weight.grad, batch_norm.weight.grad.shape)
print("\nGradiente de beta (δE/δβ):")
print(batch_norm.bias.grad, batch_norm.bias.grad.shape)

# Imprime los parámetros de la capa BatchNorm
print("\nParámetros de la capa BatchNorm:")
for name, param in batch_norm.named_parameters():
    print(name, param.data)

Gradiente de la entrada (δE/δx):
tensor([[ 2.8979e-04, -2.7798e-04, -8.7130e-04, -2.6777e-05,  4.5487e-04,
         -2.2148e-05,  1.8504e-06,  2.0966e-05, -3.1500e-02,  9.4646e-04],
        [-2.8958e-04,  2.7809e-04,  8.7110e-04,  2.6777e-05, -4.5527e-04,
          2.2137e-05, -1.8774e-06, -2.1019e-05,  3.1497e-02, -9.4629e-04]]) torch.Size([2, 10])

Gradiente de gamma (δE/δγ):
tensor([ 0.5032,  0.7106,  0.2856, -0.1302,  0.5234,  0.0537,  0.0660,  0.3042,
         0.3532, -0.1024]) torch.Size([10])

Gradiente de beta (δE/δβ):
tensor([0.7446, 0.9420, 0.9203, 0.9599, 1.2091, 0.5546, 0.9001, 1.6669, 1.3949,
        1.1233]) torch.Size([10])

Parámetros de la capa BatchNorm:
weight tensor([0.6344, 0.8494, 0.7245, 0.6110, 0.7224, 0.3230, 0.3618, 0.2283, 0.2937,
        0.6310])
bias tensor([0.0921, 0.4337, 0.4309, 0.4937, 0.4258, 0.3123, 0.4264, 0.8934, 0.9442,
        0.5018])


In [11]:
utils.check_same(x.grad.numpy(),layer_grad[0],tol=1e-5)

SUCCESS :) Arrays are equal (tolerance 1e-05)


In [12]:
utils.check_same(batch_norm.weight.grad.numpy(),layer_grad[1]['w'],tol=1e-5)

SUCCESS :) Arrays are equal (tolerance 1e-05)


In [13]:
utils.check_same(batch_norm.bias.grad.numpy(),layer_grad[1]['b'],tol=1e-5)

SUCCESS :) Arrays are equal (tolerance 1e-05)


In [14]:
samples = 100
batch_size=2
din=10 # dimensión de entrada
input_shape=(batch_size,din)

# Verificar las derivadas de un modelo de BatchNorm
# con valores aleatorios de `w`, `b`, y `x`, la entrada
layer=nn.BatchNorm(din)


utils.check_gradient.common_layer(layer,input_shape,samples=samples,tolerance=1e-5)    

BatchNorm_1 layer:
 ERROR 
δEδx
 Relative error (max):0.20005 (tolerance: 1e-05)
######################## Details: ######################## 
 Input x:
[[-0.5489551   1.80597011  1.51886562 -0.35400011 -0.82343141  0.13021495
   1.26729865  0.33276498  0.5565487  -0.21208012]
 [ 0.4562709   1.54454445 -0.23966878  0.14330773  0.25381648  0.28372536
  -1.41188888 -1.87686866 -1.01965507  0.1679423 ]]
 Input δEδy:
[[ 0.55385617 -0.53067456  1.37725748 -0.14317597  0.020316   -0.19396387
   0.13402679  0.70447407  0.66565344 -0.89842294]
 [ 1.52366378 -1.09502646  0.07922701 -0.27439657 -1.04899168 -0.07512059
  -0.74081377  0.07290724  0.40308596  1.47192937]]
 δEδx (numerical, automatic):
[[ 7.85297926e-05 -6.29592753e-04  2.80610755e-05 -1.29596637e-04
  -1.41530727e-05  1.08539892e-03  3.29570705e-07  2.62805824e-06
   6.47174430e-06  6.99965057e-04]
 [-7.85321398e-05  6.29520508e-04 -2.80606161e-05  1.29604454e-04
   1.41534645e-05 -1.08561070e-03 -3.29565849e-07 -2.62801577e-06
  -6.